In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import sys
import subprocess

# Function to install packages
def install_packages(packages):
    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
            print(f"Successfully installed {package}")
        except Exception as e:
            print(f"Error installing {package}: {e}")

# Install required packages
packages_to_install = [
    'ultralytics',
    'roboflow',
    'gradio',
    'opencv-python-headless',
    'numpy',
    'tensorflow'
]
print("Installing required packages...")
install_packages(packages_to_install)
print("Package installation complete!")

Installing required packages...
Package installation complete!


In [16]:
# Imports
import os
import cv2
import numpy as np
import gradio as gr
import tensorflow as tf
from ultralytics import YOLO
from google.colab import drive

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

# Verify the model path exists
model_path = '/content/drive/MyDrive/dental_images/best.pt'
if os.path.exists(model_path):
    print(f"Model found at: {model_path}")
else:
    print(f"Warning: Model not found at {model_path}")
    print("Please ensure your YOLO model is at the correct location in Google Drive")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!
Model found at: /content/drive/MyDrive/dental_images/best.pt


In [17]:
# Preprocessing Functions
def standardize_image(image):
    """
    Ensure image is in a consistent format:
    - Convert to 3-channel BGR if needed
    - Resize to a standard size
    - Normalize intensity
    """
    # Handle different input types
    if image is None:
        raise ValueError("Invalid image input")

    # Ensure 3 channels
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    elif image.shape[2] == 4:  # If RGBA
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2BGR)

    # Resize to a standard size while maintaining aspect ratio
    max_size = 800
    h, w = image.shape[:2]
    scale = min(max_size/w, max_size/h)
    new_w = int(w * scale)
    new_h = int(h * scale)

    image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)

    return image

def enhance_image(image):
    """
    Enhanced image preprocessing with multiple techniques
    """
    # Standardize image first
    image = standardize_image(image)

    # Convert to float32 for processing
    image = image.astype(np.float32) / 255.0

    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    # Convert to LAB color space
    lab = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_BGR2LAB)

    # Split the LAB image to different channels
    l, a, b = cv2.split(lab)

    # Apply CLAHE to L-channel
    l_clahe = clahe.apply(l)

    # Merge the CLAHE enhanced L-channel with the a and b channel
    enhanced_lab = cv2.merge((l_clahe, a, b))

    # Convert back to BGR
    enhanced_image = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

    return enhanced_image

# Test the preprocessing functions with a sample image
print("Testing preprocessing functions...")
# Create a simple test image
test_image = np.ones((300, 300, 3), dtype=np.uint8) * 127
# Add some simple shapes for visualization
cv2.rectangle(test_image, (50, 50), (250, 250), (200, 100, 100), -1)
cv2.circle(test_image, (150, 150), 50, (100, 200, 100), -1)

# Process the test image
standardized_image = standardize_image(test_image)
enhanced_image = enhance_image(test_image)

print(f"Original image shape: {test_image.shape}")
print(f"Standardized image shape: {standardized_image.shape}")
print("Preprocessing functions working correctly!")

Testing preprocessing functions...
Original image shape: (300, 300, 3)
Standardized image shape: (800, 800, 3)
Preprocessing functions working correctly!


In [18]:
def jaw_segmentation(image):
    """
    Advanced jaw segmentation that creates proper upper and lower jaw separation
    """
    # Ensure standardized image
    image = standardize_image(image)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply contrast enhancement for better feature extraction
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)

    # Apply noise reduction
    blurred = cv2.GaussianBlur(enhanced, (5, 5), 0)

    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 2
    )

    # Morphological operations to clean up the mask
    kernel = np.ones((3, 3), np.uint8)
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_CLOSE, kernel)

    # Find teeth contours
    contours, _ = cv2.findContours(
        cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # Filter contours by size to focus on teeth
    min_contour_area = 100
    teeth_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]

    if not teeth_contours:
        # Fallback if no good contours found
        h, w = gray.shape
        upper_jaw = np.zeros_like(gray)
        upper_jaw[:h//2, :] = gray[:h//2, :]

        lower_jaw = np.zeros_like(gray)
        lower_jaw[h//2:, :] = gray[h//2:, :]

        return upper_jaw, lower_jaw

    # Create a mask of detected teeth
    teeth_mask = np.zeros_like(gray)
    cv2.drawContours(teeth_mask, teeth_contours, -1, 255, -1)

    # Determine the separation curve between upper and lower jaws
    # First, find the centroid of each contour
    centroids = []
    for cnt in teeth_contours:
        M = cv2.moments(cnt)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            centroids.append((cx, cy))

    # If we can't find centroids, use fallback
    if not centroids:
        h, w = gray.shape
        upper_jaw = np.zeros_like(gray)
        upper_jaw[:h//2, :] = gray[:h//2, :]

        lower_jaw = np.zeros_like(gray)
        lower_jaw[h//2:, :] = gray[h//2:, :]

        return upper_jaw, lower_jaw

    # Sort centroids by y-coordinate
    centroids.sort(key=lambda c: c[1])

    # Determine the midpoint between upper and lower teeth
    # This approach assumes teeth are roughly in two clusters
    if len(centroids) >= 4:  # Need enough points to make a reasonable estimate
        # Calculate distances between consecutive y-coordinates
        y_diffs = [centroids[i+1][1] - centroids[i][1] for i in range(len(centroids)-1)]

        # Find the largest gap - this should be between upper and lower teeth
        max_diff_idx = y_diffs.index(max(y_diffs))
        separation_y = (centroids[max_diff_idx][1] + centroids[max_diff_idx+1][1]) // 2
    else:
        # Fallback if we don't have enough centroids
        h = gray.shape[0]
        separation_y = h // 2

    # Create masks for upper and lower jaws
    h, w = gray.shape

    # Create a smooth separation curve
    # Create a curved separation line
    curve_points = []
    for x in range(0, w, 10):
        # Create a slight curve - higher in the middle, lower at the edges
        offset = int(20 * np.sin(np.pi * x / w))
        curve_y = separation_y - offset
        curve_points.append((x, curve_y))

    # Interpolate to get a point for every x coordinate
    x_values = [p[0] for p in curve_points]
    y_values = [p[1] for p in curve_points]

    # Use np.interp for linear interpolation
    all_x = np.arange(w)
    all_y = np.interp(all_x, x_values, y_values).astype(int)

    # Create masks based on the curve
    upper_mask = np.zeros((h, w), dtype=np.uint8)
    lower_mask = np.zeros((h, w), dtype=np.uint8)

    # Fill masks based on the separation curve
    for x in range(w):
        y_sep = all_y[x]
        upper_mask[:y_sep, x] = 1
        lower_mask[y_sep:, x] = 1

    # Apply masks to the original image
    upper_jaw = cv2.bitwise_and(gray, gray, mask=upper_mask)
    lower_jaw = cv2.bitwise_and(gray, gray, mask=lower_mask)

    # Apply the teeth mask as well to highlight only teeth areas
    teeth_upper = cv2.bitwise_and(teeth_mask, teeth_mask, mask=upper_mask)
    teeth_lower = cv2.bitwise_and(teeth_mask, teeth_mask, mask=lower_mask)

    return teeth_upper, teeth_lower

# Test the jaw segmentation function
print("Testing jaw segmentation function...")
# Create a simple test image that mimics a dental X-ray
test_dental = np.zeros((400, 300), dtype=np.uint8)
# Add upper teeth (white regions in top half)
for i in range(5):
    x = 50 + i * 40
    y = 100
    cv2.circle(test_dental, (x, y), 15, 255, -1)
# Add lower teeth (white regions in bottom half)
for i in range(5):
    x = 50 + i * 40
    y = 300
    cv2.circle(test_dental, (x, y), 15, 255, -1)

# Convert to 3-channel image for processing
test_dental_bgr = cv2.cvtColor(test_dental, cv2.COLOR_GRAY2BGR)

# Apply jaw segmentation
upper_jaw, lower_jaw = jaw_segmentation(test_dental_bgr)

print(f"Upper jaw shape: {upper_jaw.shape}")
print(f"Lower jaw shape: {lower_jaw.shape}")
print("Jaw segmentation function working correctly!")

Testing jaw segmentation function...
Upper jaw shape: (800, 600)
Lower jaw shape: (800, 600)
Jaw segmentation function working correctly!


In [19]:
def detect_dental_diseases(image_path):
    """
    Robust disease detection with error handling and returns detection data
    """
    try:
        # Attempt to load model
        model = YOLO('/content/drive/MyDrive/dental_images/best.pt')
        print(f"Model loaded successfully")

        # Detect diseases
        results = model(image_path)
        print(f"Detection completed on image: {image_path}")

        # Get detection results
        detections = []
        for result in results:
            boxes = result.boxes
            for box in boxes:
                cls = int(box.cls.item())
                conf = box.conf.item()
                category = result.names[cls]
                detections.append({
                    'category': category,
                    'confidence': conf,
                    'box': box.xyxy.tolist()[0]  # Convert to list for easier handling
                })

        # Annotate image with detections
        annotated_image = results[0].plot()

        return annotated_image, detections
    except Exception as e:
        print(f"Detection error: {e}")
        # Return original image if detection fails
        if os.path.exists(image_path):
            return cv2.imread(image_path), []
        else:
            return np.zeros((300, 300, 3), dtype=np.uint8), []

def create_detection_summary(detections):
    """
    Create a summary image showing the count of different dental findings
    """
    # Count occurrences of each category
    category_counts = {}
    for detection in detections:
        category = detection['category']
        if category in category_counts:
            category_counts[category] += 1
        else:
            category_counts[category] = 1

    # Create a blank image for the summary
    height = 100 + (len(category_counts) * 30)
    height = max(height, 200)  # Ensure minimum height
    width = 400
    summary_image = np.ones((height, width, 3), dtype=np.uint8) * 255

    # Add title
    cv2.putText(summary_image, "DETECTION SUMMARY", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Add separator line
    cv2.line(summary_image, (20, 50), (width-20, 50), (0, 0, 0), 1)

    # Add category counts
    y_pos = 90
    if not category_counts:
        cv2.putText(summary_image, "No detections found", (30, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1)
    else:
        for category, count in category_counts.items():
            text = f"{category.capitalize()}: {count}"
            cv2.putText(summary_image, text, (30, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 1)
            y_pos += 30

    return summary_image

# Test the detection summary function
print("Testing detection summary function...")
# Create some sample detections
sample_detections = [
    {'category': 'cavity', 'confidence': 0.95, 'box': [10, 10, 50, 50]},
    {'category': 'cavity', 'confidence': 0.87, 'box': [60, 60, 100, 100]},
    {'category': 'gingivitis', 'confidence': 0.92, 'box': [120, 120, 180, 180]}
]

# Create a summary
summary_image = create_detection_summary(sample_detections)
print(f"Detection summary image shape: {summary_image.shape}")
print("Detection summary function working correctly!")

Testing detection summary function...
Detection summary image shape: (200, 400, 3)
Detection summary function working correctly!


In [20]:
def dental_disease_pipeline(image_path):
    """
    Comprehensive pipeline with robust error handling
    """
    try:
        # Read input image
        image = cv2.imread(image_path)

        if image is None:
            raise ValueError("Unable to read image")

        print(f"Processing image: {image_path}")
        print(f"Image shape: {image.shape}")

        # Image enhancement
        enhanced_image = enhance_image(image)
        print("Image enhancement complete")

        # Jaw segmentation
        upper_jaw, lower_jaw = jaw_segmentation(enhanced_image)
        print("Jaw segmentation complete")

        # Disease detection
        print("Starting disease detection...")
        detection_result, detections = detect_dental_diseases(image_path)
        print(f"Disease detection complete. Found {len(detections)} detections")

        # Create detection summary
        detection_summary = create_detection_summary(detections)
        print("Detection summary created")

        return {
            'Original Image': image,
            'Enhanced Image': enhanced_image,
            'Upper Jaw': upper_jaw,
            'Lower Jaw': lower_jaw,
            'Disease Detection': detection_result,
            'Detection Summary': detection_summary,
            'Raw Detections': detections
        }
    except Exception as e:
        print(f"Pipeline error: {e}")
        return None

# Create a test image path
print("Testing pipeline function...")
# Create a temp test image
test_path = '/content/test_dental.png'
test_img = np.zeros((400, 300, 3), dtype=np.uint8)
# Add some shapes
cv2.rectangle(test_img, (50, 50), (250, 150), (200, 200, 200), -1)
cv2.circle(test_img, (150, 250), 50, (180, 180, 180), -1)
cv2.imwrite(test_path, test_img)

print(f"Created test image at: {test_path}")
print("Note: The disease detection may show warnings if the model isn't found.")
print("This is expected during testing if you haven't yet uploaded your model.")

# Test the pipeline
results = dental_disease_pipeline(test_path)
if results:
    print("Pipeline test successful!")
else:
    print("Pipeline test failed. Check the errors above.")

Testing pipeline function...
Created test image at: /content/test_dental.png
Note: The disease detection may show warnings if the model isn't found.
This is expected during testing if you haven't yet uploaded your model.
Processing image: /content/test_dental.png
Image shape: (400, 300, 3)
Image enhancement complete
Jaw segmentation complete
Starting disease detection...
Model loaded successfully

image 1/1 /content/test_dental.png: 640x480 (no detections), 94.1ms
Speed: 3.0ms preprocess, 94.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)
Detection completed on image: /content/test_dental.png
Disease detection complete. Found 0 detections
Detection summary created
Pipeline test successful!


In [21]:
# Gradio Interface
def gradio_interface(image):
    """
    Gradio interface with comprehensive error handling
    """
    # Temporary save path
    temp_path = '/content/temp_upload.png'

    try:
        # Save uploaded image
        cv2.imwrite(temp_path, image)
        print(f"Received image uploaded to: {temp_path}")

        # Process image
        print("Starting dental disease pipeline...")
        results = dental_disease_pipeline(temp_path)

        if results is None:
            raise ValueError("Processing failed")

        print("Processing complete, returning results")
        return [
            results['Original Image'],
            results['Enhanced Image'],
            results['Upper Jaw'],
            results['Lower Jaw'],
            results['Disease Detection'],
            results['Detection Summary']
        ]
    except Exception as e:
        print(f"Interface error: {e}")
        # Create error images
        error_image = np.zeros((300, 300, 3), dtype=np.uint8)
        cv2.putText(error_image, str(e), (10, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        return [error_image] * 6

print("Gradio interface function defined successfully")

Gradio interface function defined successfully


In [22]:
# Create Gradio Interface
print("Creating Gradio interface...")
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.Image(label="Original Image"),
        gr.Image(label="Enhanced Image"),
        gr.Image(label="Upper Jaw"),
        gr.Image(label="Lower Jaw"),
        gr.Image(label="Disease Detection"),
        gr.Image(label="Detection Summary")
    ],
    title="Dental Disease Detection",
    description="Upload a dental X-ray to detect diseases and segment jaw regions.",
    allow_flagging="never"
)

# Launch the interface
print("Launching Gradio interface. A link will be provided below:")
iface.launch(debug=True)

Creating Gradio interface...


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Launching Gradio interface. A link will be provided below:
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://66ef1583faf532f3ab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Received image uploaded to: /content/temp_upload.png
Starting dental disease pipeline...
Processing image: /content/temp_upload.png
Image shape: (840, 1615, 3)
Image enhancement complete
Jaw segmentation complete
Starting disease detection...
Model loaded successfully

image 1/1 /content/temp_upload.png: 352x640 1 Filling, 2 Root Canal Treatments, 1 Bone Loss, 72.4ms
Speed: 1.9ms preprocess, 72.4ms inference, 2.5ms postprocess per image at shape (1, 3, 352, 640)
Detection completed on image: /content/temp_upload.png
Disease detection complete. Found 4 detections
Detection summary created
Processing complete, returning results
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://66ef1583faf532f3ab.gradio.live
